In [ ]:
from matplotlib import pyplot as plt
from nltk.tokenize import sent_tokenize
import pandas as pd
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
import numpy as np
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#df=pd.read_csv('results1.csv')
df=pd.read_csv('incubationperiod.csv')

summ=df.head(100)
summ.head()


In [ ]:
print( postprocessing(summ.summary[0]))

In [ ]:
def remove_stopwords(sen):     
    sen_new = " ".join([i for i in sen if i not in stop_words])          
    return sen_new

In [ ]:
i=1
sentences = [] 
for s in summ['summary']: 
     if i<5:              
        for x in(sent_tokenize(s)) :
            sentences.append(sent_tokenize(x))
        i=i+1
print(sentences)    
corpus = [y for x in sentences for y in x ]
#print(corpus)


clean_sentences = pd.Series(corpus).str.replace("[^a-zA-Z]", " ") 
        # make alphabets lowercase 

clean_sentences = [s.lower() for s in clean_sentences]
#clean_sentences = [remove_stopwords(r.split()) for r in corpus]


In [ ]:

# Extract word vectors 
word_embeddings = {} 
f = open(r"C:\meriem\Mémoire\glove6b\glove.6B.100d.txt", encoding='utf-8') 
for line in f: 
    values = line.split() 
    word = values[0] 
    coefs = np.asarray(values[1:], dtype='float32')    
    word_embeddings[word] = coefs 
f.close()

In [ ]:
sentence_vectors = [] 
for i in clean_sentences: 
    if len(i) != 0: 
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001) 
    else: 
        v = np.zeros((100,)) 
    sentence_vectors.append(v)

In [ ]:
sim_mat = np.zeros([len(corpus), len(corpus)])
for i in range(len(corpus)): 
      for j in range(len(corpus)): 
        if i != j: 
              sim_mat[i][j] = cosine_similarity (sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
print( sim_mat)

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt

nx_graph = nx.from_numpy_array(sim_mat) 
nx.draw(nx_graph,with_labels=True)
scores = nx.pagerank(nx_graph)

In [ ]:

ranked_sentences = sorted(((scores[i],s) for i,s in 
                           enumerate(corpus)), reverse=True)
# Extract top 10 sentences as the summary 
for i in range(12): 
      print(ranked_sentences[i][1])

In [ ]:
import re
def postprocessing(text):
    # capitalize the text
    text = text.capitalize()
    # '2 , 3' -> '2,3'
    text = re.sub(r'(\d) \, (\d)', "\g<1>,\g<2>", text)
    # full stop
    text = re.sub(r' (\.|\!|\?) (\w)', lambda pat: pat.groups()[0]+" "+pat.groups()[1].upper(), text)
    # full stop at the end
    text = re.sub(r' (\.|\!|\?)$', "\g<1>", text)
    # comma
    text = re.sub(r' (\,|\;|\:) (\w)', "\g<1> \g<2>", text)
    # - | \ / @ and _ (e.g. 2 - 3  -> 2-3)
    text = re.sub(r'(\w) (\-|\||\/|\\|\@\_) (\w)', "\g<1>\g<2>\g<3>", text)
    # parenthesis
    text = re.sub(r'(\(|\[|\{) ([^\(\)]+) (\)|\]|\})', "\g<1>\g<2>\g<3>", text)
    # "" e.g. " word "  -> "word"
    text = re.sub(r'(\") ([^\"]+) (\")', "\g<1>\g<2>\g<3>", text)
    # apostrophe  e.g. l ' a  -> l'a
    text = re.sub(r'(\w) (\') (\w)', "\g<1>\g<2>\g<3>", text)
    # '3 %' ->  '3%'
    text = re.sub(r'(\d) \%', "\g<1>%", text)
    # '# word'  -> '#word'
    text = re.sub(r'\# (\w)', "#\g<1>", text)
    # https and doi
    text = re.sub(r'(https|doi) : ', "\g<1>:", text)
    return text

In [ ]:

#ranked_sentences = sorted(((scores[i],s) for i,s in  enumerate(corpus)), reverse=True)
# Extract top 10 sentences as the summary 
for i in range(12): 
      print(postprocessing(ranked_sentences[i][1]))